In [ ]:
import pandas as pd
import numpy as np
import random
import datetime

# Cargar el dataset de películas
df_movies = pd.read_csv('imdb_clean.csv')

# Paso 1: Generar Usuarios
def generar_usuarios(num_usuarios):
    users = []
    for user_id in range(1, num_usuarios + 1):
        username = f"user_{user_id}"
        password = f"pass_{user_id}"
        users.append([user_id, username, password])
    return pd.DataFrame(users, columns=['user_id', 'username', 'password'])

num_usuarios = 1000
df_users = generar_usuarios(num_usuarios)

# Paso 2: Generar Interacciones coherentes (películas mostradas, vistas, etc.)
def generar_interacciones(num_interacciones, num_usuarios, df_movies):
    interacciones = []
    tipos_interaccion = ['shown', 'view']  # Tipos de interacción relevantes
    emociones = ['Happy', 'Down', 'Excited', 'Relaxed', 'Sweet', 'Scared', 'Inspired']
    
    for _ in range(num_interacciones):
        user_id = random.randint(1, num_usuarios)
        emotion = random.choice(emociones)
        
        # Filtrar películas por emoción seleccionada
        peliculas_filtradas = df_movies[df_movies['emotions'].apply(lambda x: emotion in x)]
        
        if not peliculas_filtradas.empty:
            movie_id = peliculas_filtradas.sample(1).index[0]  # Seleccionar una película al azar que corresponda a la emoción
            interaction_type = random.choice(tipos_interaccion)
            date = datetime.datetime.now() - datetime.timedelta(days=random.randint(0, 365))
            interacciones.append([user_id, movie_id, emotion, interaction_type, date])
        
    return pd.DataFrame(interacciones, columns=['user_id', 'movie_id', 'emotion', 'interaction_type', 'date'])

num_interacciones = 20000
df_interacciones = generar_interacciones(num_interacciones, num_usuarios, df_movies)

# Paso 3: Generar Favoritos coherentes (deben haber sido "shown" o "view")
def generar_favoritos(df_interacciones, num_favoritos):
    favoritos = []
    interacciones_validas = df_interacciones[df_interacciones['interaction_type'].isin(['shown', 'view'])]

    for _ in range(num_favoritos):
        fila = interacciones_validas.sample(1).iloc[0]  # Seleccionar una interacción aleatoria
        user_id = fila['user_id']
        movie_id = fila['movie_id']
        date_added = datetime.datetime.now() - datetime.timedelta(days=random.randint(0, 365))
        favoritos.append([user_id, movie_id, date_added])
    
    return pd.DataFrame(favoritos, columns=['user_id', 'movie_id', 'date_added'])

num_favoritos = 5000
df_favoritos = generar_favoritos(df_interacciones, num_favoritos)

# Paso 4: Generar Ratings coherentes (solo de favoritos)
def generar_ratings(df_favoritos, num_ratings):
    ratings = []
    
    for _ in range(num_ratings):
        fila = df_favoritos.sample(1).iloc[0]  # Seleccionar un favorito aleatorio
        user_id = fila['user_id']
        movie_id = fila['movie_id']
        rating = random.randint(1, 10)  # Rating entre 1 y 10
        date = datetime.datetime.now() - datetime.timedelta(days=random.randint(0, 365))
        ratings.append([user_id, movie_id, rating, date])

    return pd.DataFrame(ratings, columns=['user_id', 'movie_id', 'rating', 'date'])

num_ratings = 8000
df_ratings = generar_ratings(df_favoritos, num_ratings)

# Guardar los dataframes generados en archivos CSV para luego cargarlos en la base de datos
df_users.to_csv('users_sinteticos.csv', index=False)
df_interacciones.to_csv('interacciones_sinteticas.csv', index=False)
df_favoritos.to_csv('favoritos_sinteticos.csv', index=False)
df_ratings.to_csv('ratings_sinteticos.csv', index=False)

print("Generación de datos sintéticos completada.")
